# HPO for Customer Churn Prediction with XGBoost
_**본 노트북은 1번 노트북에서 작업한 고객이탈예측에 하이퍼파라미터 최적화 작업을 수행합니다.**_


---
#### 주의 : 1번 노트북 작업을 완료후 실행해야 합니다.

---

## Contents

1. [Setup](#Setup)
1. [Train with HPO](#Train-with-HPO)
1. [Analyze result of HPO job](#Analyze-result-of-HPO-job)
1. [Analyze result of HPO parameters](#Analyze-result-of-HPO-parameters)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Compare results](#Compare-results)

---

## Setup

SageMaker SDK를 로드하고 초기변수를 설정합니다. (1번 노트북의 첫단계와 동일합니다.)


In [8]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/DEMO-xgboost-churn"

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

필요한 파이썬 라이브러리를 import 합니다. 

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

1번 노트북에서 사용한 변수를 리스토어합니다.

In [10]:
%store -r s3_input_train s3_input_validation test_data predictions

In [65]:
print(s3_input_train.config.values())

dict_values([{'S3DataSource': {'S3DataType': 'S3Prefix', 'S3Uri': 's3://sagemaker-ap-northeast-1-308961792850/sagemaker/DEMO-xgboost-churn/train', 'S3DataDistributionType': 'FullyReplicated'}}, 'csv'])


---
## Train with HPO

HPO 작업을 위해 XGBoost Estimator를 선언합니다. 먼저 XGBoost 컨테이너의 위치를 가져옵니다.

In [11]:
container = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "latest")
display(container)

'501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/xgboost:latest'

Estimator의 설정은 1번 단계와 거의 동일하나 몇가지가 달라졌습니다.
- hyperparameter를 세팅할 때 max_depth, etc, min_child_weight 값을 설정하지 않았습니다.
- xgb.fit()을 바로 실행하지 않았습니다. (대신 바로 다음 스텝에서 Hyperparameter tuning 작업의 매개변수로 전달됩니다.)

In [66]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    gamma=4,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
)


아래는 1번 노트북에서 사용한 코드입니다. 어떤 부분이 달려졌는지 비교해 보시기 바랍니다.

```
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    silent=0,
    objective="binary:logistic",
    num_round=100,
)

xgb.fit({"train": s3_input_train, "validation": s3_input_validation})
```



### SageMaker tunner 선언

SageMaker Hypter parameter tunner를 선언합니다.
- objective_metric_name : HPO작업에서 최적화할 목표 매트릭을 설정
- hyperparameter_ranges : 탐색하고자하는 하이퍼파라미터와 해당 값의 범위를 지정
- max_jobs : 총 실행할 작업의 회수
- max_parallel_jobs : 동시에 실행할 작업의 회수

In [17]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

objective_metric_name = "validation:auc"

hyperparameter_ranges = {
    "eta": ContinuousParameter(0, 1),
    "min_child_weight": ContinuousParameter(1, 10),
    "max_depth": IntegerParameter(1, 10),
}

tuner = HyperparameterTuner(
    xgb, objective_metric_name, hyperparameter_ranges, max_jobs=20, max_parallel_jobs=3
)


### HPO 작업 실행
다음 셀을 실행하고 SageMaker 콘솔로 이동하여 Hyperparameter tuning jobs 메뉴를 클릭합니다. 작업이 제출되고 실행되는 것을 모니터링할 수 있습니다. (max_jobs와 max_parallel_jobs 설정이 HPO 작업실행에 어떤 영향을 미치는지 확인해 보세요.)

In [20]:
%%time
tuner.fit({"train": s3_input_train, "validation": s3_input_validation})

............................................................................................................................................................................................................................................................................................................................................................!


## Analyze result of HPO job

HPO 작업이 완료되면 다음 코드를 이용하여 결과를 탐색해 봅니다.

분석 코드는 아래 SageMaker 공식예제로부터 일부 변수값 등을 변형하였습니다.
- https://github.com/aws/amazon-sagemaker-examples/blob/master/hyperparameter_tuning/analyze_results/HPO_Analyze_TuningJob_Results.ipynb


In [21]:
tuning_job_name=tuner.latest_tuning_job.job_name

boto3.client("sagemaker").describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)["HyperParameterTuningJobStatus"]


'Completed'

boto3 SDK를 이용하여 작업결과를 가져오고 `tuning_job_result` 로 저장하였습니다. 해당 오브젝트를 살펴보면 실행한 작업의 수 등 HPO작업의 설정 사항과 상태 등을 확인할 수 있습니다.


In [67]:
sage_client = boto3.Session().client("sagemaker")
# run this cell to check current status of hyperparameter tuning job
tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
    HyperParameterTuningJobName=tuning_job_name
)

status = tuning_job_result["HyperParameterTuningJobStatus"]
if status != "Completed":
    print("Reminder: the tuning job has not been completed.")

job_count = tuning_job_result["TrainingJobStatusCounters"]["Completed"]
print("%d training jobs have completed" % job_count)

is_minimize = (
    tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["Type"] != "Maximize"
)
objective_name = tuning_job_result["HyperParameterTuningJobConfig"]["HyperParameterTuningJobObjective"]["MetricName"]


20 training jobs have completed


20번의 탐색 중 가장 높은 성능을 보인 작업을 살펴봅니다. 작업실행 시간등의 로그와 함께 탐색대상으로 지정했던 `eta`, `max_depth`, `min_child_weight` 에 대한 최적값을 확인할 수 있습니다. (`TunedHyperParameters`항목을 확인합니다.)

In [35]:
from pprint import pprint

if tuning_job_result.get("BestTrainingJob", None):
    print("Best model found so far:")
    pprint(tuning_job_result["BestTrainingJob"])
else:
    print("No training jobs have reported results yet.")


Best model found so far:
{'CreationTime': datetime.datetime(2021, 7, 4, 6, 45, 48, tzinfo=tzlocal()),
 'FinalHyperParameterTuningJobObjectiveMetric': {'MetricName': 'validation:auc',
                                                 'Value': 0.9860320091247559},
 'ObjectiveStatus': 'Succeeded',
 'TrainingEndTime': datetime.datetime(2021, 7, 4, 6, 49, 55, tzinfo=tzlocal()),
 'TrainingJobArn': 'arn:aws:sagemaker:ap-northeast-1:308961792850:training-job/xgboost-210704-0625-017-27f78aa1',
 'TrainingJobName': 'xgboost-210704-0625-017-27f78aa1',
 'TrainingJobStatus': 'Completed',
 'TrainingStartTime': datetime.datetime(2021, 7, 4, 6, 48, 30, tzinfo=tzlocal()),
 'TunedHyperParameters': {'eta': '0.09557658233972222',
                          'max_depth': '10',
                          'min_child_weight': '8.868466891754721'}}


## Analyze result of HPO parameters

SageMaker의 HyperparameterTuningJobAnalytics를 이용하여 하이퍼파라미터 탐색작업에서 실행한 실험의 결과를 살펴봅니다.

In [51]:
tuning_result = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)
full_df = tuning_result.dataframe()

if len(full_df) > 0:
    df = full_df[full_df["FinalObjectiveValue"] > -float("inf")]
    if len(df) > 0:
        df = df.sort_values("FinalObjectiveValue", ascending=is_minimize)
        print("Number of training jobs with valid objective: %d" % len(df))
        print({"lowest": min(df["FinalObjectiveValue"]), "highest": max(df["FinalObjectiveValue"])})
        pd.set_option("display.max_colwidth", -1)  # Don't truncate TrainingJobName
    else:
        print("No training jobs have reported valid results yet.")

df

Number of training jobs with valid objective: 20
{'lowest': 0.9763540029525757, 'highest': 0.9860320091247559}


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


,eta,max_depth,min_child_weight,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds
3,0.095577,10.0,8.868467,xgboost-210704-0625-017-27f78aa1,Completed,0.986032,2021-07-04 06:48:30+00:00,2021-07-04 06:49:55+00:00,85.0
13,0.372165,6.0,5.109769,xgboost-210704-0625-007-f2e6155f,Completed,0.985996,2021-07-04 06:35:37+00:00,2021-07-04 06:36:35+00:00,58.0
4,0.089938,8.0,7.952855,xgboost-210704-0625-016-8971e77c,Completed,0.985283,2021-07-04 06:48:22+00:00,2021-07-04 06:49:08+00:00,46.0
2,0.089084,10.0,6.947606,xgboost-210704-0625-018-4c1f4abf,Completed,0.985147,2021-07-04 06:48:29+00:00,2021-07-04 06:49:36+00:00,67.0
17,0.535116,4.0,5.268480,xgboost-210704-0625-003-cd562b3e,Completed,0.984967,2021-07-04 06:27:44+00:00,2021-07-04 06:28:50+00:00,66.0
18,0.722963,8.0,7.836629,xgboost-210704-0625-002-1f4cc2cb,Completed,0.984823,2021-07-04 06:27:45+00:00,2021-07-04 06:28:34+00:00,49.0
0,0.110877,10.0,9.048467,xgboost-210704-0625-020-72def18a,Completed,0.984771,2021-07-04 06:52:43+00:00,2021-07-04 06:53:30+00:00,47.0
7,0.086734,6.0,1.057188,xgboost-210704-0625-013-c1788ff7,Completed,0.984699,2021-07-04 06:44:11+00:00,2021-07-04 06:45:21+00:00,70.0
5,0.329765,9.0,1.940763,xgboost-210704-0625-015-e89dbe4d,Completed,0.984351,2021-07-04 06:44:14+00:00,2021-07-04 06:45:02+00:00,48.0
11,0.746569,8.0,1.116063,xgboost-210704-0625-009-7e5e42c6,Completed,0.983334,2021-07-04 06:35:51+00:00,2021-07-04 06:36:38+00:00,47.0


그래프를 통해 작업별 실행한 결과를 확인해 보겠습니다. 그래프상의 관측값에 마우스를 위치시키면 해당 실험이 하이퍼파라미터 범위 중 어떤 값을 사용하였고 그 때의 매트릭이 어떻게 관측되었는지 확인할 수 있습니다.

In [68]:
import bokeh
import bokeh.io

bokeh.io.output_notebook()
from bokeh.plotting import figure, show
from bokeh.models import HoverTool


class HoverHelper:
    def __init__(self, tuning_analytics):
        self.tuner = tuning_analytics

    def hovertool(self):
        tooltips = [
            ("FinalObjectiveValue", "@FinalObjectiveValue"),
            ("TrainingJobName", "@TrainingJobName"),
        ]
        for k in self.tuner.tuning_ranges.keys():
            tooltips.append((k, "@{%s}" % k))

        ht = HoverTool(tooltips=tooltips)
        return ht

    def tools(self, standard_tools="pan,crosshair,wheel_zoom,zoom_in,zoom_out,undo,reset"):
        return [self.hovertool(), standard_tools]


hover = HoverHelper(tuning_result)

p = figure(plot_width=900, plot_height=400, tools=hover.tools(), x_axis_type="datetime")
p.circle(source=df, x="TrainingStartTime", y="FinalObjectiveValue")
show(p)

Loading BokehJS ...

유사한 방법으로 탐색대상으로 설정한 하이퍼파라미터별로 매트릭의 결과에 어떤 영향을 미쳤는지 확인해 봅니다.

In [69]:
ranges = tuning_result.tuning_ranges
figures = []
for hp_name, hp_range in ranges.items():
    categorical_args = {}
    if hp_range.get("Values"):
        # This is marked as categorical.  Check if all options are actually numbers.
        def is_num(x):
            try:
                float(x)
                return 1
            except:
                return 0

        vals = hp_range["Values"]
        if sum([is_num(x) for x in vals]) == len(vals):
            # Bokeh has issues plotting a "categorical" range that's actually numeric, so plot as numeric
            print("Hyperparameter %s is tuned as categorical, but all values are numeric" % hp_name)
        else:
            # Set up extra options for plotting categoricals.  A bit tricky when they're actually numbers.
            categorical_args["x_range"] = vals

    # Now plot it
    p = figure(
        plot_width=500,
        plot_height=500,
        title="Objective vs %s" % hp_name,
        tools=hover.tools(),
        x_axis_label=hp_name,
        y_axis_label=objective_name,
        **categorical_args,
    )
    p.circle(source=df, x=hp_name, y="FinalObjectiveValue")
    figures.append(p)
show(bokeh.layouts.Column(*figures))

---
## Host

HPO작업을 통해 찾아낸 최적의 하이퍼파라미터를 이용하여 모델을 생성하고 호스팅 엔드포인트로 배포합니다. 


In [70]:
hpo_predictor = tuner.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)


2021-07-04 06:49:55 Starting - Preparing the instances for training
2021-07-04 06:49:55 Downloading - Downloading input data
2021-07-04 06:49:55 Training - Training image download completed. Training in progress.
2021-07-04 06:49:55 Uploading - Uploading generated training model
2021-07-04 06:49:55 Completed - Training job completed
-------------!

### Evaluate

1번 노트북에서 실행했던 것과 유사한 방법으로 테스트데이터에 대한 예측을 실행하과 1번 작업의 결과와 비교해 보겠습니다.


In [71]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ""
    for array in split_array:
        predictions = ",".join([predictions, hpo_predictor.predict(array).decode("utf-8")])

    return np.fromstring(predictions[1:], sep=",")


predictions_hpo = predict(test_data.to_numpy()[:, 1:])

### Compare results

이번에는 sklearn에서 제공하는 classification_report 기능을 이용하여 결과를 확인하겠습니다. 편의상 0.5를 컷오프로 지정하였습니다. 


In [72]:
from sklearn.metrics import classification_report
print(classification_report(test_data.iloc[:, 0], np.round(predictions)))

              precision    recall  f1-score   support

           0       0.97      0.94      0.96       253
           1       0.94      0.97      0.96       247

    accuracy                           0.96       500
   macro avg       0.96      0.96      0.96       500
weighted avg       0.96      0.96      0.96       500



다음은 HPO로 찾아낸 하이퍼파라미터로 작업한 결과입니다. 전체적으로 이전 모델의 성능과 유사하나 0번 분류에 대한 precision과 recall 값이 조금 상승하였습니다.  

In [73]:
from sklearn.metrics import classification_report
print(classification_report(test_data.iloc[:, 0], np.round(predictions_hpo)))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       253
           1       0.95      0.97      0.96       247

    accuracy                           0.96       500
   macro avg       0.96      0.96      0.96       500
weighted avg       0.96      0.96      0.96       500



### (Optional) Clean-up

모든 작업을 완료하였다면 추가 요금발생을 막기 위해 아래 셀을 실행하십시오. 아래 코드는 생성한 호스팅 엔드포인트를 제거합니다. 

In [41]:
hpo_predictor.delete_endpoint()